# Overview

This notebook walks through how PyDough can be leveraged to construct "WHAT-IF" queries. It highlights how PyDough can be write to solve common, intermediate sub-problems, enabling fast iteration and development.

We believe that this approach is more composable and scalable. Rather than focusing on a single query to answer a question at hand, building question components allows more proportional scaling relative to question complexity. The PyDough team is optimistic that such an approach can make humans more productive and LLMs more accurate.

The majority of this notebook will be focused on composing a single question piece by piece. At some places we will also deviate from this question to emphasize the powerful PyDough features that enable this type of experimentation.

In [ ]:
%load_ext pydough.jupyter_extensions

In [ ]:
import pydough
import pandas as pd

In [ ]:
# Setup demo metadata
pydough.active_session.load_metadata_graph("../metadata/tpch_demo_graph.json", "TPCH");
pydough.active_session.connect_database("sqlite", database="../../tpch.db");
# Avoid scientific notation
pd.options.display.float_format = '{:.6f}'.format

## Schema Context

For this demo we will be working in an example crafted to match the schema for the TPC-H benchmark. The actual data for this benchmark is generated in SQLite using the standard TPC-H data generation tools. The underlying schema of this data matches this example image from [TPC Benchmark H Standard Specification](https://www.tpc.org/tpc_documents_current_versions/pdf/tpc-h_v2.17.1.pdf).

![TPC-H schema from the Specification Document as of December 12, 2024](../images/tpc_h_schema.png)

## Problem Statement

We are going to seek to **identify the 5 lines in our entire order sheet that represent the smallest percentage of their corresponding order's revenue**. 

Here is a possible SQL query that could we be used to answer this question and in constrast here is the corresponding PyDough.

### SQL

```SQL
Select
    (L_EXTENDED_PRICE * (1 - L_DISCOUNT)) / order_total as revenue_ratio,
    orderkey,
    l_linenumber as line_number
from lines
inner join (
    select
        o_orderkey as order_key,
        SUM(L_EXTENDED_PRICE * (1 - L_DISCOUNT)) as order_total
    from orders
    inner join lines
    on l_orderkey = o_orderkey
    group by o_orderkey
)
on l_orderkey = order_key
order by revenue_ratio ASC, order_key DESC, line_number DESC
LIMIT 5
```

### PyDough

```python
price_def = extended_price*(1-discount)
orders(total_line_price=SUM(lines(line_price=price_def).line_price)).lines(
    revenue_ratio=price_def / BACK(1).total_price, 
    order_key=order_key, 
    line_number=line_number
).TOP_K(5, by=(revenue_ratio.ASC(), order_key.DESC(), line_number.DESC()))
```

The example SQL requires a nested subquery to answer, which can easily become complicated and unmanagable as questions scale in size. Similarly, example query requires explicitly join decisions, especially deciding on join keys and `INNER` vs `LEFT` join. In this particular example **because we know the exact constraints of the TPC-H setup** we can verify that the provided query is correct. However, in a production setting, or attempting to transition a query across two different workspaces, these assumptions could change, leading to subtle incorrect results that are extremely difficult to debug.

In constrast, PyDough handles the join information and the correctness requirements by specifying metadata.

## Metadata

PyDough's metadata dictates which tables are available and how table information can be combined. This eliminates the need to worry about subtle correctness questions, such as `LEFT` vs `INNER` join or the exact join keys. This also means that the same code can be deployed on slightly different metadata and these correctness considerations will be handled automatically.

We will not explore the metadata in detail in this notebook, but we will briefly address a couple basic concepts. We also have a separate metadata notebook to explore all of these in more detail.

In [ ]:
%%pydough

print(pydough.explain(pydough.active_session.metadata))

This tells us the collections that we can immediately access. These match the TPC-H schema, but have been updated with human readable names. We can then look at lines to learn more information. 

In [ ]:
%%pydough

print(pydough.explain(pydough.active_session.metadata["lines"], verbose=True))

Lines shows us a couple notable pieces of information:
* The underlying table in the database. Our metadata is generated without any data migration.
* Human readable names for each property.
* The other collects we can reach. These automatically encode our join information.

## PyDough Solution

While we showed one version of the PyDough solution above, we are going to take a different approach to show how one might generate this query in a "WHAT-IF" manner. This approach will be a longer solution, but this is helpful to enable faster iteration.

We are opting to demonstrate this because:
1. We believe this is a good representation for how PyDough can be leveraged to gradually handle increasing question complexity.
2. We believe this reflects an investigative approach to query generation, where someone may understand at a high-level what needs to be done, but not necessary that "path" to get there.

To do this we will first need to define revenue. Here we will say that the revenue is the price information after removing any discounts.

In [ ]:
%%pydough

price_def = extended_price*(1-discount)

This might seem shocking. We have defined `price_def` out of nowhere using an `extended_price` and `discount`. What has actually happened here is that we have generated what is called a `Contextless Expression`. This fundamental building block is the key to PyDough's composability.

On its own this expression doesn't mean anything. In fact if we inspect this object in regular PyDough we will see that PyDough itself has a lot of questions.

In [ ]:
price_def

As you see, PyDough now knows that this expression is composed of `extended_price` and `discount`, but it doesn't know **WHICH** `extended_price` and `discount`. To ultimately develop a legal PyDough statement, we will need to bind uses of this expression to a context that can access `extended_price` and `discount`.

This might seem very minor, but this allows us to define definitions up front, allowing reuse in vastly different contexts.

Now our schema is flexible, but this price defintion is actually tied to the lines collection. Let's now combine incorporate lines.

In [ ]:
%%pydough

line_price = lines(line_price=price_def).line_price

At this stage our expression is more complete (we know it is associated with lines), but its actually still contextless. Let's extend this by now asking for something more concrete, the SUM.

In [ ]:
%%pydough

total_price = SUM(line_price)
total_price

Now this expression is more meaningful. If we assign this statement to global context, our actual TPCH graph, then we can compute the total price across all lines.

In [ ]:
%%pydough

pydough.to_df(TPCH(total_line_price=total_price))

In practice though, this may not solve our core question. Instead, we may want to apply a different **context**, say for example total_price for each order. We can instead represent that as follows.

In [ ]:
%%pydough

order_total_price = orders(total_line_price=total_price)
pydough.to_df(order_total_price)

Notice that are able to reuse the exact same code, but by swapping the context we can ultimately modify the semantics. This makes testing the underlying behavior much more scalable. To ask is this statement correct, we can instead compose our question to ask:
* Is this underlying expression correct?
* Is this context correct?

Since these can be verify independently, we can develop greater confidence in our question since it arises from composable building blocks. We could also generate selected contexts to build clear testing.

In [ ]:
%%pydough

# Compute the sum of the first 5 line numbers, which can be known for testing.
top_five_lines = lines.TOP_K(5, by=(line_number.ASC(), order_key.ASC()))
top_five_line_price = TPCH(total_price=SUM(top_five_lines(line_price=price_def).line_price))
pydough.to_df(top_five_line_price)

Now let's return to extending our question. Building able to compute order sums is great, but we care about results per line. As a result, now we can even extend our orders to an additional context within lines. We will once again define more defintions. Our ratio definition will now ask us to propagate our previous `total_price` price that we computed and compare it to the result of `price_def`.

In [ ]:
%%pydough

ratio = price_def / BACK(1).total_price

Now we will build our final question by moving from the context of the orders to the context of orders and lines together. Since Orders -> Lines is a Many to One relationship, this places us in the context of lines with some additional information.

For actually fully solving our prior question, we will compute the ratio and then select 5 smallest ratio value, breaking ties with a combination of the order number and line number.

In [ ]:
%%pydough

line_ratios = order_total_price.lines(revenue_ratio=ratio, order_key=order_key, line_number=line_number)
lowest_ratios = line_ratios.TOP_K(5, by=(revenue_ratio.ASC(), order_key.DESC(), line_number.DESC()))

In [ ]:
pydough.to_df(lowest_ratios)

Now we have resolved to solution to our underlying question. We can save this result to a Python variable, as the output is already a Pandas DataFrame. Alternatively, we could expand on our question to collect more information, such as filtering by a minimum number of entries. For completeness we will show that example where we only **consider lines that are part of orders with more than 3 lines**. 

In [ ]:
%%pydough

total_lines = COUNT(lines)

In [ ]:
%%pydough

order_total_price = orders(total_line_price=total_price, line_count=total_lines)

In [ ]:
%%pydough

line_ratios = order_total_price.lines(revenue_ratio=ratio, line_count=BACK(1).line_count, order_key=order_key, line_number=line_number)
filtered_ratios = line_ratios.WHERE(line_count > 3)(revenue_ratio, order_key, line_number)

lowest_ratios = filtered_ratios.TOP_K(5, by=(revenue_ratio.ASC(), order_key.DESC(), line_number.DESC()))
highest_ratios = filtered_ratios.TOP_K(5, by=(revenue_ratio.DESC(), order_key.DESC(), line_number.DESC()))

In [ ]:
%%pydough

pydough.to_df(lowest_ratios)

In [ ]:
%%pydough

pydough.to_df(highest_ratios)

The lowest ratio doesn't seem to change, most likely because these already arise from situations with multiple orders. However, now we can meaningfully ask about the highest ratios without being dominated by single line orders.